In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
import os

In [2]:
spark = SparkSession.builder \
    .appName("CSV to PostgreSQL") \
    .getOrCreate()

24/11/20 16:32:16 WARN Utils: Your hostname, spark-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/11/20 16:32:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/20 16:32:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Lendo CSV
df = spark.read.csv("./output/sor/ipeadata.csv", header=True, inferSchema=True)

In [4]:
df = df.withColumn("valdata", to_date(col("valdata")))
df = df.withColumn("valvalor", col("valvalor").cast("float"))

In [5]:
df.dtypes

[('sercodigo', 'string'),
 ('valdata', 'date'),
 ('valvalor', 'float'),
 ('nivnome', 'string'),
 ('tercodigo', 'string')]

In [2]:
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
schema = os.getenv("DB_SCHEMA")
table = os.getenv("DB_TABLE")

In [23]:
# Define PostgreSQL connection properties
url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

# Write DataFrame to PostgreSQL
df.write.jdbc(url=url, table=f"{schema}.{table}", mode="overwrite", properties=properties)

# Stop the Spark session
spark.stop()